In [1]:
from PIL import Image
import torch
import numpy as np 
from glob import glob
from tqdm import tqdm

In [2]:
import os
import platform
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Using {device} as device")

Using cuda as device


### Setup paths

In [4]:
pti_experiments_path = f"{DATA_PATH}/Models/PTI/experiments/"
pti_input_images_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/pti_images/sample/"
generated_images_output_path = f"{DATA_PATH}/Generated_Images/PTI/"

In [5]:
input_images = glob(f"{pti_input_images_path}*.jpg")
input_images = [elem.split('/')[-1] for elem in input_images]
len(input_images)

500

In [6]:
os.chdir(f'{ROOT_PATH}/stylegan2-ada-pytorch/')
def invert_from_sku(sku):
    # Get correct generator:
    model_path = f"{pti_experiments_path}checkpoints/model_whole_sample_{sku.split('.')[0]}.pt"
    G_PTI = torch.load(model_path, map_location=torch.device('cpu'))
    G_PTI = G_PTI.to(device)

    # Load corresponding latent embedding
    latent_path = f"{pti_experiments_path}embeddings/zalando_germany/PTI/{sku.split('.')[0]}/0.pt"
    latent = torch.load(latent_path, map_location=torch.device('cpu'))
    latent = latent.to(device)

    #  Create image from finetuned-PTI model
    img = G_PTI.synthesis(latent, force_fp32=True)
    img_perm = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img_pti = Image.fromarray(img_perm[0].cpu().numpy(), 'RGB')
    return img_pti

In [7]:
for sku in tqdm(input_images):
    img = invert_from_sku(sku)
    img.save(f"{generated_images_output_path}{sku}")

100%|██████████| 500/500 [10:08<00:00,  1.22s/it]
